In [1]:
import pandas as pd
import random
from collections import defaultdict
from tqdm import tqdm


In [2]:
# Load Elo and matchup data
elo_df = pd.read_csv("Final_Elo_Ratings_with_Full_Name_Matching.csv")
matchups_df = pd.read_csv("French_Open_Round_1_Matchups.csv")

# Create Elo lookup dictionary
elo_lookup = dict(zip(elo_df['Player Name'], elo_df['Elo']))

# Build round 1 matchups list
round1_matchups = list(zip(matchups_df['Player 1'], matchups_df['Player 2']))


In [3]:
# Elo win probability function
def win_prob(elo1, elo2):
    return 1 / (1 + 10 ** ((elo2 - elo1) / 400))

# Elo fetch fallback (in case a name is missing)
def get_elo(name):
    return elo_lookup.get(name, 1600)

# Simulate a single tournament from a starting round
def simulate_tournament(matchups):
    winners = []
    for p1, p2 in matchups:
        e1, e2 = get_elo(p1), get_elo(p2)
        prob1 = win_prob(e1, e2)
        winner = p1 if random.random() < prob1 else p2
        winners.append(winner)
    while len(winners) > 1:
        next_round = []
        for i in range(0, len(winners)-1, 2):
            p1, p2 = winners[i], winners[i+1]
            e1, e2 = get_elo(p1), get_elo(p2)
            prob1 = win_prob(e1, e2)
            winner = p1 if random.random() < prob1 else p2
            next_round.append(winner)
        winners = next_round
    return winners[0]


In [4]:
# Run the tournament simulation 10,000 times
win_counts = defaultdict(int)

for _ in tqdm(range(10000), desc="Simulating Tournament"):
    champion = simulate_tournament(round1_matchups)
    win_counts[champion] += 1

# Convert results to a DataFrame
results = pd.DataFrame(win_counts.items(), columns=["Player", "Championships"])
results["Win %"] = results["Championships"] / 10000
results = results.sort_values(by="Win %", ascending=False).reset_index(drop=True)

# Show top 15
results.head(15)


Simulating Tournament: 100%|██████████| 10000/10000 [00:01<00:00, 8974.46it/s]


,Player,Championships,Win %
0,C. Alcaraz,3293,0.3293
1,J. Sinner,1766,0.1766
2,N. Djokovic,1116,0.1116
3,L. Musetti,665,0.0665
4,A. Zverev,648,0.0648
5,C. Ruud,510,0.0510
6,J. Draper,279,0.0279
7,S. Tsitsipas,271,0.0271
8,D. Medvedev,170,0.0170
9,H. Rune,169,0.0169
